

3. ***XGBoost***


In [0]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

**Mounting G drive**


> You need to authenticate to GD in order to get the AWID dataset with the scripts.



In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


**Get the resources**

> Get all necessary resources from Google Drive, this works after the authentification.these resources are compressed to save some time.


In [0]:
!cp -rn "/content/drive/My Drive/AWID/AWID-CLS-R-Trn_with_headers.zip" .
!cp -rn "/content/drive/My Drive/AWID/AWID-CLS-R-Tst_with_headers.zip" .
!cp -rn "/content/drive/My Drive/Mahamdi's PFE/for Collab/utils.py" .
!cp -rn "/content/drive/My Drive/Mahamdi's PFE/for Collab/main.py" .
!cp -rn "/content/drive/My Drive/Mahamdi's PFE/for Collab/imports.py" .
!cp -rn "/content/drive/My Drive/Mahamdi's PFE/for Collab/class_report.py" .

!unzip AWID-CLS-R-Trn_with_headers.zip 
!unzip AWID-CLS-R-Tst_with_headers.zip 
!rm *.zip
!rm -rf sample_data/

Archive:  AWID-CLS-R-Trn_with_headers.zip
  inflating: AWID-CLS-R-Trn/AWID-CLS-R-Trn_with_headers.csv  
Archive:  AWID-CLS-R-Tst_with_headers.zip
  inflating: AWID-CLS-R-Tst/AWID-CLS-R-Tst_with_headers.csv  


***Imports***


> Import all necessary libraries.



In [0]:
from imports import *

***Read the data and replace the '?' string in the Pandas DataFrame with a NumPy NaN value***

In [0]:
train_path = '/content/AWID-CLS-R-Trn/AWID-CLS-R-Trn_with_headers.csv'
test_path = '/content/AWID-CLS-R-Tst/AWID-CLS-R-Tst_with_headers.csv'
train_df_raw = pd.read_csv(train_path, low_memory=False)
test_df_raw = pd.read_csv(test_path, low_memory=False)
train_df_raw.replace('?', np.nan, inplace=True)
test_df_raw.replace('?', np.nan, inplace=True)


**Drop these colomns**


> Each one of the dropped columns has a Nan rate of 1.
> Note that we need to drop them from both of training and validation sets.


In [0]:
del test_df_raw['frame.dlt']
del test_df_raw['wlan.qos.buf_state_indicated']
del test_df_raw['radiotap.mactime']
del test_df_raw['wlan.ba.bm']
del test_df_raw['wlan.ba.control.ackpolicy']
del test_df_raw['wlan.ba.control.cbitmap']
del test_df_raw['wlan.ba.control.multitid']
del test_df_raw['wlan.bar.compressed.tidinfo']
del test_df_raw['wlan.bar.type']
del test_df_raw['wlan_mgt.fixed.fragment']
del test_df_raw['wlan_mgt.fixed.sequence']
del test_df_raw['wlan_mgt.tcprep.link_mrg']
del test_df_raw['wlan_mgt.tcprep.trsmt_pow']


del train_df_raw['frame.dlt']
del train_df_raw['wlan.qos.buf_state_indicated']
del train_df_raw['radiotap.mactime']
del train_df_raw['wlan.ba.bm']
del train_df_raw['wlan.ba.control.ackpolicy']
del train_df_raw['wlan.ba.control.cbitmap']
del train_df_raw['wlan.ba.control.multitid']
del train_df_raw['wlan.bar.compressed.tidinfo']
del train_df_raw['wlan.bar.type']
del train_df_raw['wlan_mgt.fixed.fragment']
del train_df_raw['wlan_mgt.fixed.sequence']
del train_df_raw['wlan_mgt.tcprep.link_mrg']
del train_df_raw['wlan_mgt.tcprep.trsmt_pow']


***Data pre-processing***


> We need to treat the data before passing it to our model.



***Change any columns of strings in a panda's dataframe to a column of categorical values.***
 

> This applies the changes inplace.




In [0]:
str_to_cat(train_df_raw)

***Changes any columns of strings in df into categorical variables using trn as a template for the category codes***
 

> This applies the changes inplace too.




In [0]:
apply_cats(test_df_raw,train_df_raw)

***We're still not quite done !***
 
> For instance we have lots of missing values, which we can't pass directly to the model.




***We call  the function proc_df() to process a pandas dataframe !***
 

*   *proc_df*  takes a data frame df and splits off the response variable in our case the *class* colomn, and changes the df into an entirely numeric dataframe. For each column of df which is not in skip_flds nor in ignore_flds, na values are replaced by the median value of the column.
*    Three other funtions are called inside this function:
    *    ***numericalize()*** : Changes the column col from a categorical type to it's integer codes.
    *    ***fix_missing()*** : Fill missing data in a column of df with the median, and add a {name}_na column 
    which specifies if the data was missing.
    *    ***scale_vars()*** : Scale tha data.



In [0]:
x_train, y_train , nas , mapper = proc_df(train_df_raw, 'class', do_scale=True)
x_valid, y_valid , nas , _ = proc_df(test_df_raw,'class', na_dict=nas, do_scale=True, mapper=mapper)

***Classification with XGBoost***

Instantiate and train the model

In [0]:
model = XGBClassifier(n_estimators=100)
%time model.fit(x_train, y_train)

Evaluate the model:

In [0]:
# making predictions on the testing set 
y_pred = model.predict(x_valid)
predict= [round(value) for value in y_pred]

# predict = gnb.predict(x_valid)
CM = confusion_matrix(y_valid, predict)
CR = classification_report (y_valid,predict)
report_with_auc = class_report(y_true=y_valid, y_pred=y_pred,y_score=model.predict_proba(x_valid))
print("************************the original set *************************")
print("=== Confusion Matrix ===\n",CM)
print("\n=== Report with classification report  ===\n\n",CR)
print("\n=== Report with auc  ===\n\n",report_with_auc)

***features selection***
> We are going to test our model in different sets.

> Want to know more about these sets? The details are in the report.





In [0]:
features2 = ['radiotap.datarate', 'wlan.fc.type_subtype', 'wlan.seq','wlan_mgt.fixed.capabilities.preamble','wlan_mgt.fixed.timestamp','wlan_mgt.fixed.beacon','wlan_mgt.tim.dtim_period','data.len','class']
features3 = ['frame.time_epoch' , 'frame.time_relative' , 'radiotap.datarate' ,'wlan.fc.type_subtype' ,'wlan.fc.ds' ,'wlan.fc.retry' ,'wlan.fc.protected' ,'wlan.ta' ,'wlan.seq' ,'wlan_mgt.fixed.capabilities.ess' ,'wlan_mgt.fixed.capabilities.preamble' ,'wlan_mgt.fixed.capabilities.short_slot_time' ,'wlan_mgt.fixed.timestamp' ,'wlan_mgt.fixed.beacon' ,'wlan_mgt.fixed.auth.alg' ,'wlan_mgt.fixed.auth_seq' ,'wlan_mgt.tim.dtim_period' ,'wlan_mgt.rsn.akms.type' ,'wlan.wep.key' ,'data.len' ,'class']
features4 = ['frame.time_epoch','frame.time_delta','frame.time_delta_displayed','frame.time_relative','frame.len','frame.cap_len','wlan.fc.subtype','wlan.fc.ds','wlan.duration','wlan.ra','wlan.da','wlan.ta','wlan.sa','wlan.seq','wlan.wep.iv','wlan.wep.icv','data.len','class']

# set 2
train2 = train_df_raw[features2]
test2 = test_df_raw[features2]  
str_to_cat(train2)
apply_cats(test2,train2)
x_valid2 , y_valid2 , nas2 , mapper2 = proc_df(test2, 'class' , do_scale = True)
x_train2 , y_train2 , nas2 , _ = proc_df(train2, 'class',na_dict=nas2 , do_scale = True , mapper = mapper2)

# set 3
train3 = train_df_raw[features3]
test3 = test_df_raw[features3] 
str_to_cat(train3)
apply_cats(test3,train3)
x3,y3,nas3 = proc_df(train3, 'class')
x_valid3 , y_valid3 , nas3 , mapper3 = proc_df(test3,'class' , do_scale = True)
x_train3 , y_train3 , nas3 , _ = proc_df(train3, 'class',na_dict=nas3 , do_scale = True , mapper = mapper3)

# set 4
train4 = train_df_raw[features4]
test4 = test_df_raw[features4] 
str_to_cat(train4)
apply_cats(test4,train4)
x4,y4,nas4 = proc_df(train4, 'class')
x_valid4 , y_valid4 , nas4 , mapper4 = proc_df(test4,'class' , do_scale = True)
x_train4 , y_train4 , nas4 , _ = proc_df(train4, 'class',na_dict=nas4 , do_scale = True , mapper = mapper4)



In [0]:
# set2
model2 = XGBClassifier( gamma=0,learning_rate=0.1, max_delta_step=0, max_depth=3,
              min_child_weight=1, n_estimators=100, n_jobs=-1,
              nthread=4, objective='multi:softprob', random_state=0)
%time model2.fit(x_train2, y_train2) 
  
# making predictions on the testing set 
predict2 = model2.predict(x_valid2)
CM2 = confusion_matrix(y_valid2, predict2)
CR2 = classification_report (y_valid2,predict2)
report_with_auc2 = class_report(y_true=y_valid2, y_pred=predict2,y_score=model2.predict_proba(x_valid2))
print("************************set2 *************************")
print("=== Confusion Matrix ===\n",CM2)
print("\n=== Report with auc  ===\n\n",report_with_auc2)

CPU times: user 6min 8s, sys: 703 ms, total: 6min 9s
Wall time: 1min 39s
************************set2 *************************
=== Confusion Matrix ===
 [[  1076      0      0   7021]
 [     0   1467      0  18612]
 [     0    379  13524   2779]
 [    37    470  15170 515107]]

=== Report with auc  ===

              precision    recall  f1-score   support      pred       AUC
0             0.966757  0.132889  0.233659    8097.0    1113.0  0.929222
1             0.633420  0.073061  0.131011   20079.0    2316.0  0.869454
2             0.471318  0.810694  0.596086   16682.0   28694.0  0.978882
3             0.947726  0.970464  0.958960  530784.0  543519.0  0.926922
avg / total   0.923224  0.922751  0.909362  575642.0  575642.0  0.991408


In [0]:
# set3
model3 = XGBClassifier()
%time model3.fit(x_train3, y_train3) 
  
# making predictions on the testing set 
predict3 = model3.predict(x_valid3)
CM3 = confusion_matrix(y_valid3, predict3)
CR3 = classification_report (y_valid3,predict3)
report_with_auc3 = class_report(y_true=y_valid3, y_pred=predict3,y_score=model3.predict_proba(x_valid3))
print("************************set3 *************************")
print("=== Confusion Matrix ===\n",CM3)
print("\n=== Report with auc  ===\n\n",report_with_auc3)

In [0]:
# set4
model4 = XGBClassifier()
%time model4.fit(x_train4, y_train4) 

# making predictions on the testing set 
predict4 = model4.predict(x_valid4)
CM4 = confusion_matrix(y_valid4, predict4)
CR4 = classification_report (y_valid4,predict4)
report_with_auc4 = class_report(y_true=y_valid4, y_pred=predict4,y_score=model4.predict_proba(x_valid4))
print("************************set4 *************************")
print("=== Confusion Matrix ===\n",CM4)
print("\n=== Report with auc  ===\n\n",report_with_auc4)

CPU times: user 8min 18s, sys: 583 ms, total: 8min 19s
Wall time: 8min 19s
************************set4 *************************
=== Confusion Matrix ===
 [[  4412      0      0   3685]
 [     0   1254      0  18825]
 [     0      0  13811   2871]
 [    13      0      0 530771]]

=== Report with auc  ===

              precision    recall  f1-score   support      pred       AUC
0             0.997062  0.544893  0.704680    8097.0    4425.0  0.968926
1             1.000000  0.062453  0.117564   20079.0    1254.0  0.907365
2             1.000000  0.827898  0.905847   16682.0   13811.0  0.997375
3             0.954363  0.999976  0.976637  530784.0  556152.0  0.950279
avg / total   0.957878  0.955886  0.940795  575642.0  575642.0  0.995310


***Save the models***

> With the ***dill*** framework,which is a utility for serialization of python objects.



In [0]:
os.makedirs("XGBoost_Models", exist_ok=True)

dill.dump(model, open("XGBoost_Models/XGBoost.obj", "wb")) 
dill.dump(model2, open("XGBoost_Models/XGBoost1.obj", "wb"))
dill.dump(model3, open("XGBoost_Models/XGBoost2.obj", "wb"))
dill.dump(model4, open("XGBoost_Models/XGBoost3.obj", "wb"))

In [0]:
!zip -r XGBoost_Models.zip XGBoost_Models
!cp -f XGBoost_Models.zip "/content/drive/My Drive/Mahamdi's PFE/Models"

  adding: XGBoost_Models/ (stored 0%)
  adding: XGBoost_Models/XGBoost3.obj (deflated 69%)
  adding: XGBoost_Models/XGBoost1.obj (deflated 71%)
